# Plotly-Dash+RAPIDSによる人口推移の可視化

## パッケージのインストール

In [2]:
import pandas as pd
import geopandas as gpd
import ast,os,random
pd.set_option('display.float_format','{:.1f}'.format)
import warnings
warnings.filterwarnings('ignore')
import cudf
import cupy as cp
import numpy as np
import time
import math
import pickle
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import cuxfilter


table = pq.read_table('../input/training/185/total_population_dataset.parquet')

In [3]:
# GPUのチェック
!nvidia-smi

Sun Oct  1 13:46:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.236.01   Driver Version: 450.236.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:1F:00.0 Off |                    0 |
| N/A   40C    P0    37W / 250W |    309MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## データフレームの概要
2020年の総人口と，2010年からの移住に関する情報が記されたデータ

Net
- 1: Inward Migration（内部移住）
- 0: Stationary（定住）
- -1: Outward Migration（外部移住）

Race
- 0: All
- 1: White
- 2: African American
- 3: American Indian
- 4: Asian alone
- 5: Native Hawaiian
- 6: Other Race alone
- 7: Two or More


County \
Mappings for counties can be found in id2county.pkl file from root directory. \
 county(群）のID
 
 easting/northing \
 緯度と経度

In [4]:
cdf = cudf.DataFrame.from_arrow(table)
cdf.head()

,easting,northing,race,net,county
0,-9626792.0,3825189.8,1,0,0
1,-9626832.0,3825073.8,1,0,0
2,-9627101.0,3825153.5,1,0,0
3,-9627149.0,3825322.8,1,0,0
4,-9627159.0,3825334.8,1,0,0


## cuxfilterの準備

In [5]:
# cuXfilterのデータフレームを作成する
cux_df = cuxfilter.DataFrame(cdf)

## ダッシュボードの作成

In [6]:
# 人種
label_map = {
    0: 'All',
    1: 'White',
    2: 'African American',
    3: 'American Indian',
    4: 'Asian alone',
    5: 'Native Hawaiian',
    6: 'Other Race alone',
    7: 'Two or More'    
}

# 人口分布図
chart0 = cuxfilter.charts.scatter(x='easting', y='northing', 
                                  tile_provider='CartoLight', title = 'Population Distribution of Individuals', )
# 移動の記録
chart1 = cuxfilter.charts.bar('net',  title = 'Move Between 2010 and 2020')

# フィルターを作成
chart2 = cuxfilter.charts.multi_select('race', label_map=label_map)


d = cux_df.dashboard(charts=[chart0, chart1], sidebar=[chart2], title='dashboard_example', layout = cuxfilter.layouts.feature_and_base, theme=cuxfilter.themes.light, data_size_widget=True)
d.app()

Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=280)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=381254348)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='race_multi_select', width=400)
            [0] MultiSelect(height=200, options={'All': 0, 'White': 1, ...}, sizing_mode='stretch_both', value=[''], width=400)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=11, nrows=6, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='Population Distribution o..., width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')
        [1] Card(sizing_mode='stretch_both', title='Move Between 2..., width=400)
            [0] Bokeh(Figure, sizing_mode='stretch_both')
            [1] RangeSlider(sizing_mode='scale_width', start=-1, step=1, value=(-1, 1), value_start=-1, width=400)